In [1]:
import tensorflow as tf
import os
import numpy as np

USE_GPU = True

if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
from tensorflow.keras.preprocessing.image import ImageDataGenerator   

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = (x_train - 127.5) - 1
x_test = (x_test - 127.5) - 1

datagen = ImageDataGenerator(rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True) 

# replica = 10

# x_train_len = x_train.shape[0]
# x_train_image_shape = x_train.shape[1:]

# x_train_argumented = np.empty((x_train_len*replica,*x_train_image_shape))
# y_train_argumented = np.empty((x_train_len*replica,*y_train.shape[1:]))

# for index in range(replica):
#     print(index)
#     x_train_argumented[x_train_len*index:x_train_len*(index+1)] = next(datagen.flow(x_train, batch_size=x_train_len))
#     y_train_argumented[x_train_len*index:x_train_len*(index+1)] = y_train
    
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)

In [3]:
def learning_rate_scheduler(
    epoch, learning_rate, schedule_list=None, exponent=0.1, warm_up=True, warm_up_epoch=10
):
    step = 0
    if warm_up and epoch < warm_up_epoch:
        new_learning_rate = learning_rate * ((epoch + 1) / warm_up_epoch)
    else:
        if schedule_list is None:
            schedule_list = [25, 50, 100, 175, 250, 400]
        for step, target_epoch in enumerate(schedule_list):
            if target_epoch > epoch:
                break
            else:
                continue
        new_learning_rate = learning_rate * (exponent ** (step))

    return new_learning_rate

In [4]:
from gan_module.model.build_model import build_classifier
from tensorflow.keras.optimizers import Nadam
input_img_shape = (32, 32, 3)
optimizer = Nadam(1e-3, clipnorm=1)
num_class = 10
loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
model = build_classifier(
    input_img_shape=input_img_shape,
    classfier_power=64,
    depth=3,
    num_class=num_class,
)

In [5]:
from tensorflow.keras.callbacks import History 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam

check_point = ModelCheckpoint(
    "./temp/weights_{epoch:02d}_{loss:.4f}.hdf5", 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='min')

reduceLROnPlat = ReduceLROnPlateau(
      monitor="val_loss",
      factor=0.1,
      patience=20,
      verbose=1,
      mode="auto",
      min_delta=0.0001,
      cooldown=5,
      min_lr=1e-7
)
lr_scheduler = LearningRateScheduler(learning_rate_scheduler)


history = History()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [6]:
nb_epoch = 400
batch_size = 32
model.fit(x=x_train,
          y=y_train_one_hot,
            epochs=nb_epoch,
            batch_size=batch_size,
            validation_split=0.1,
            shuffle = True,
            callbacks = [check_point, reduceLROnPlat, history])

Epoch 1/400
1407/1407 [==============================] - 450s 320ms/step - loss: 3.7805 - accuracy: 0.3495 - val_loss: 0.3267 - val_accuracy: 0.3776
Epoch 2/400
1407/1407 [==============================] - 432s 307ms/step - loss: 0.3152 - accuracy: 0.4315 - val_loss: 0.3315 - val_accuracy: 0.3892
Epoch 3/400
1407/1407 [==============================] - 439s 312ms/step - loss: 0.3069 - accuracy: 0.4702 - val_loss: 0.3472 - val_accuracy: 0.3948
Epoch 4/400
1407/1407 [==============================] - 422s 300ms/step - loss: 0.3018 - accuracy: 0.4941 - val_loss: 0.3076 - val_accuracy: 0.4826
Epoch 5/400
1407/1407 [==============================] - 410s 291ms/step - loss: 0.2988 - accuracy: 0.5138 - val_loss: 0.3121 - val_accuracy: 0.4980
Epoch 6/400
1407/1407 [==============================] - 407s 289ms/step - loss: 0.2959 - accuracy: 0.5269 - val_loss: 0.3141 - val_accuracy: 0.4680
Epoch 7/400
1407/1407 [==============================] - 407s 289ms/step - loss: 0.2942 - accuracy: 0.5359

KeyboardInterrupt: 

In [ ]:
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),